In [2]:
import json
import pandas as pd
import numpy as np
import glob
import os
from courier_cleaning import *

pd.set_option('display.max_columns', 1000)

# Read in session.jsonl file

In [79]:
def add_keypress(events, rec_type, rec_stop_type):
    rec_evs = events.query('type == @rec_type | type == @rec_stop_type')
    rec_evs_index = rec_evs.index
    
    for i in range(len(rec_evs)-1):
        
        curr_row = rec_evs.iloc[i]
        if curr_row.type != rec_type:
            continue
        next_row = rec_evs.iloc[i+1]

        curr_time = curr_row.mstime
        next_time = next_row.mstime

        EFR_candidates = events.query('type == "keypress" & \
                                       mstime > @curr_time & mstime < @next_time')
        if len(EFR_candidates) > 0:
            response_time = EFR_candidates.mstime.values[0]

            events.at[rec_evs_index[i], "keypress"] = True
            events.at[rec_evs_index[i], "keypress_mstime"] = response_time
        
        # if no correct key press happened, check for additional key presses
        else:
            next_candidates = events.query('type == "key press/release" & \
                                            mstime > @curr_time & mstime < @next_time')
            display(next_candidates)
            if len(next_candidates) > 0:
                responses = next_candidates["key code"].unique()
                responses = responses[~np.isnan(responses)]
                print(responses)
                response_time = next_candidates.mstime.values[0]
                
                events.at[rec_evs_index[i], "keypress_keycode"] = responses.tolist()
                events.at[rec_evs_index[i], "keypress_mstime"] = response_time
    
    return events

In [3]:
with open('/scratch/new_courier_pilot/dbpool.txt', "r") as f:
    wordpool = [w.strip().upper() for w in f.readlines()]

data_dir = "/scratch/new_courier_pilot/"
sub_dirs = []

# get subject level directory info
for root, dirs, files in os.walk(data_dir):
    for name in files:
        if name.endswith("session.jsonl"):
            sub_dirs.append(root)
            
print(sub_dirs)


for sub_dir in sub_dirs:
    _, _, _, sub, sess_txt = sub_dir.split("/")
    sess = int(sess_txt[-1])
    print("subject {}, sess {}".format(sub, sess))
    
    # check if the data is already saved in data_dir
    file_name = "{}_sess_{}.csv".format(sub, sess)
    file_dir = os.path.join(data_dir, file_name)
#     if os.path.exists(file_dir):
#         print("already saved")
#         continue
    
    break
    
    sub_data = []
    # locate session.jsonl
    sess_dir = os.path.join(data_dir, sub_dir, "session.jsonl")
    for line in open(sess_dir, "r"):
        # replace this specific entry to empty string
        if '"point condition":SerialPosition,' in line:
            line = line.replace('"point condition":SerialPosition,', '')
        elif '"point condition":SpatialPosition,' in line:
            line = line.replace('"point condition":SpatialPosition,', '')
        elif '"point condition":Random,' in line:
            line = line.replace('"point condition":Random,', '')
            
        data_dict = json.loads(line)
        sub_data.append(data_dict)
    
    # create dataframe
    for d in sub_data:
        for k,v in d["data"].items():
            d[k] = v
    df = pd.DataFrame(sub_data)
        
    column_names = ["type", "time", "trial number", "item name", "item", "serial position", "store", "store name", "store position", 
                "player position", "positionX", "positionZ", "correct direction (degrees)", "pointed direction (degrees)", "response", "key code"]
    df = df[column_names]
    df = df.rename(columns={
                            'trial number': 'trial',
                            'time': 'mstime',
                            'serial position': 'serialpos',
                            'item': 'cued_item',
                            'store': 'cued_store',
                            'store name': 'store',
                            'item name': 'item'
                            })
    df["subject"] = sub
    df["session"] = sess
    df["recalled"] = -999
    df["intrusion"] = -999

    # change the type name to match lab's data frame formatting
    df = df.replace({'type': {'object presentation begins': 'WORD', 
                                  'free recall': 'REC_WORD',
                                  'object recall recording start': 'REC_START',
                                  'object recall recording stop': 'REC_STOP',
                                  'cued recall recording stop': 'CUED_REC_STOP',
                                  'cued recall': 'CUED_REC_WORD',
                                  'final store recall': 'FSR_REC_WORD',
                                  'final store recall recording stop': 'FSR_REC_STOP',
                                  'final object recall': 'FFR_REC_WORD',
                                  'final object recall recording stop': 'FFR_REC_STOP'
                                 }
                        })
    
    ###############################################################################################
    # FREE RECALL
    ###############################################################################################
    recall_df = pd.DataFrame(columns = df.columns)
    rec_starts = df.query('type == "REC_START"')
    # first REC_START will indicate practice recall stage for session 0
    if sess == 0:
        rec_starts = rec_starts[1:]
        
    count = 0
    for i, row in rec_starts.iterrows():
        rec_start_time = row.mstime

        # let's load appropriate .ann files
        annotation_file = "{}.ann".format(count)
        annotation_dir = os.path.join(data_dir, sub_dir, annotation_file)
        with open(annotation_dir, "r") as file:
            lines = file.readlines()
            for line in lines:
                if line[0] != "#":
                    recall_info = line.strip("\n").split("\t")

                    if len(recall_info) == 3:
                        rectime = float(recall_info[0]) + rec_start_time
                        item = recall_info[2].lower()
                        itemno = recall_info[1]

                        recall_df = recall_df.append({"subject":sub,
                                                      "session":sess,
                                                      "type":"REC_WORD", 
                                                      "mstime":rectime, 
                                                      "trial":count, 
                                                      "item":item, 
                                                      "itemno":itemno
                                                     }, ignore_index=True)
        count += 1

    # then, fill out the blanks
    recall_df["store"] = recall_df["store"].astype(str)
    recall_df["store position"] = recall_df["store position"].astype(str)

    for i, row in recall_df.iterrows():
        word_evs = df[(df.trial == row.trial) & (df.type == "WORD")]

        row_item = row["item"]
        recall_word = word_evs.query('item == @row_item')

        if len(recall_word) != 0:        
            serialpos = recall_word["serialpos"].values[0]
            store = recall_word["store"].values[0]
            store_position = recall_word["store position"].values[0]

            recall_df.at[i, "serialpos"] = serialpos
            recall_df.at[i, "store"] = store
            recall_df.at[i, "store position"] = store_position

        else:
            recall_df.at[i, "serialpos"] = -999
    
    ###############################################################################################
    # CUED RECALL
    ###############################################################################################
    cued_rec_start = df.query('type == "start cued recall"').mstime.values
    if sess == 0:
        cued_rec_start = cued_rec_start[1]
    else:
        cued_rec_start = cued_rec_start[0]
    
    cued_rec_events = df.query('type == "cued recall recording start" & mstime >= @cued_rec_start')
    cued_recall_df = pd.DataFrame(columns = df.columns)

    for i, row in cued_rec_events.iterrows():
        trial = int(row.trial)
        cued_item = row.cued_item
        cued_store = row.cued_store
        cued_time = row.mstime

        word_evs = df.query('type == "WORD" & subject == @sub & trial == @trial')
        stores = word_evs.store.unique()

        if cued_store not in stores:
            continue

        # locate the .ann file
        annotation_file = "{}-{}.ann".format(trial, cued_store)
        annotation_dir = os.path.join(data_dir, sub_dir, annotation_file)
        with open(annotation_dir, "r") as file:
            lines = file.readlines()
            for line in lines:
                if line[0] != "#":
                    recall_info = line.strip("\n").split("\t")

                    if len(recall_info) == 3:
                        rectime = float(recall_info[0]) + cued_time
                        item = recall_info[2].lower()
                        itemno = recall_info[1]

                        cued_recall_df = cued_recall_df.append({"subject":sub,
                                                                "session":sess,
                                                                "type": "CUED_REC_WORD", 
                                                                "mstime": rectime, 
                                                                "trial": trial, 
                                                                "item": item, 
                                                                "itemno": itemno, 
                                                                "cued_item": cued_item, 
                                                                "cued_store": cued_store 
                                                                }, ignore_index=True)
                        
    # then, fill out the blanks
    cued_recall_df["store"] = cued_recall_df["store"].astype(str)
    cued_recall_df["store position"] = cued_recall_df["store position"].astype(str)

    for i, row in cued_recall_df.iterrows():
        word_evs = df[(df.trial == row.trial) & (df.type == "WORD")]

        row_item = row["item"]
        recall_word = word_evs.query('item == @row_item')

        if len(recall_word) != 0:        
            serialpos = recall_word["serialpos"].values[0]
            store = recall_word["store"].values[0]
            store_position = recall_word["store position"].values[0]

            cued_recall_df.at[i, "serialpos"] = serialpos
            cued_recall_df.at[i, "store"] = store
            cued_recall_df.at[i, "store position"] = store_position

        else:
            cued_recall_df.at[i, "serialpos"] = -999
            
    ###############################################################################################
    # FINAL STORE RECALL
    ###############################################################################################
    FFR_store_df = pd.DataFrame(columns = df.columns)
    FFR_store_evs = df.query('type == "final store recall recording start"')
    if len(FFR_store_evs) != 0:
        FFR_store_start = FFR_store_evs.mstime.values[0]
    
        # let's load appropriate .ann files
        annotation_dir = os.path.join(data_dir, sub_dir, "final store-0.ann")
        with open(annotation_dir, "r") as file:
            lines = file.readlines()
            for line in lines:
                if line[0] != "#":
                    recall_info = line.strip("\n").split("\t")

                    if len(recall_info) == 3:
                        rectime = float(recall_info[0]) + FFR_store_start
                        item = recall_info[2].lower()
                        itemno = recall_info[1]

                        FFR_store_df = FFR_store_df.append({"subject":sub,
                                                            "session":sess,
                                                            "type":"FFR_STORE", 
                                                            "mstime":rectime, 
                                                            "item":item, 
                                                            "itemno":itemno, 
                                                           }, ignore_index=True)
    
    ###############################################################################################
    # FINAL ITEM RECALL
    ###############################################################################################
    FFR_item_df = pd.DataFrame(columns = df.columns)
    FFR_item_evs = df.query('type == "final object recall recording start"')
    if len(FFR_item_evs) != 0:
        FFR_item_start = FFR_item_evs.mstime.values[0]

        # let's load appropriate .ann files
        annotation_dir = os.path.join(data_dir, sub_dir, "final free-0.ann")
        with open(annotation_dir, "r") as file:
            lines = file.readlines()
            for line in lines:
                if line[0] != "#":
                    recall_info = line.strip("\n").split("\t")

                    if len(recall_info) == 3:
                        rectime = float(recall_info[0]) + FFR_item_start
                        item = recall_info[2].lower()
                        itemno = recall_info[1]

                        FFR_item_df = FFR_item_df.append({"subject":sub,
                                                          "session":sess,
                                                          "type":"FFR_ITEM", 
                                                          "mstime":rectime, 
                                                          "item":item, 
                                                          "itemno":itemno
                                                         }, ignore_index=True)

        # then, fill out the blanks
        FFR_item_df["store"] = FFR_item_df["store"].astype(str)
        FFR_item_df["store position"] = FFR_item_df["store position"].astype(str)

        for i, row in FFR_item_df.iterrows():
            word_evs = df.query('type == "WORD"')

            row_item = row["item"]
            recall_word = word_evs.query('item == @row_item')

            if len(recall_word) != 0:        
                serialpos = recall_word["serialpos"].values[0]
                store = recall_word["store"].values[0]
                store_position = recall_word["store position"].values[0]

                FFR_item_df.at[i, "serialpos"] = serialpos
                FFR_item_df.at[i, "store"] = store
                FFR_item_df.at[i, "store position"] = store_position

            else:
                FFR_item_df.at[i, "serialpos"] = -999
            
    ###############################################################################################
    # ARRANGE
    ###############################################################################################
    # now put it in right place
    tmp = df.copy()
    tmp = tmp.append(recall_df)
    tmp = tmp.append(cued_recall_df)
    tmp = tmp.append(FFR_store_df)
    tmp = tmp.append(FFR_item_df)
    tmp = tmp.sort_values('mstime').reset_index(drop=True)
    
    tmp[["trial", "serialpos", "recalled", "itemno", "intrusion"]] = tmp[["trial", "serialpos", "recalled", "itemno", "intrusion"]].fillna(-999)
    tmp = tmp.astype({"trial":int, "serialpos":int, "recalled":int, "itemno":int, "intrusion":int})
    
    tmp = add_recalled(tmp)
    tmp = add_itemno(tmp, wordpool)
    tmp = add_intrusion(tmp)
    
    ###############################################################################################
    # add EFR presses
    ###############################################################################################
    tmp["keypress_code"] = np.nan
    tmp = tmp.astype({"keypress_code":object})
    
    tmp = add_keypress(tmp, "REC_WORD", "REC_STOP")
    tmp = add_keypress(tmp, "CUED_REC_WORD", "CUED_REC_STOP")
    tmp = add_keypress(tmp, "FSR_REC_WORD", "FSR_REC_STOP")
    tmp = add_keypress(tmp, "FFR_REC_WORD", "FFR_REC_STOP")
    

['/scratch/new_courier_pilot/', '/scratch/new_courier_pilot/PILOT1/session_0', '/scratch/new_courier_pilot/PILOT2/session_0', '/scratch/new_courier_pilot/PILOT3/session_0', '/scratch/new_courier_pilot/PILOT4/session_1', '/scratch/new_courier_pilot/PILOT4/session_0', '/scratch/new_courier_pilot/PILOT5/session_1', '/scratch/new_courier_pilot/PILOT5/session_0', '/scratch/new_courier_pilot/PILOT6/session_1', '/scratch/new_courier_pilot/PILOT6/session_0']


ValueError: not enough values to unpack (expected 5, got 4)

In [83]:
a = tmp.query('type == "CUED_REC_WORD"')
a

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress_code,keypress,keypress_mstime,keypress_keycode
11955,CUED_REC_WORD,1.652297e+12,0,decals,decals,8,bike shop,bike shop,"(14.49, 2.27, -33.74)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,66,NaN,NaN,NaN,NaN
12025,CUED_REC_WORD,1.652297e+12,0,pastries,pastries,2,cafe,cafe,"(36.74, -1.91, 59.06)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,153,NaN,True,1.652297e+12,NaN
12092,CUED_REC_WORD,1.652297e+12,0,anchovies,anchovies,3,pizzeria,pizzeria,"(-14.25, 0.00, -12.89)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,6,NaN,NaN,NaN,NaN
12197,CUED_REC_WORD,1.652297e+12,0,<>,watch,-999,jewelry store,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,-1,NaN,NaN,NaN,NaN
12225,CUED_REC_WORD,1.652297e+12,0,guinea_pig,guinea_pig,12,pet store,pet store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,96,NaN,NaN,NaN,NaN
12289,CUED_REC_WORD,1.652297e+12,0,floor_mat,floor_mat,7,gym,gym,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,85,NaN,True,1.652297e+12,NaN
12371,CUED_REC_WORD,1.652297e+12,0,markers,markers,9,craft shop,craft shop,"(35.40, 0.00, -33.11)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,128,NaN,NaN,NaN,NaN
12475,CUED_REC_WORD,1.652297e+12,0,<>,towels,-999,barber shop,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,-1,NaN,NaN,NaN,NaN
12543,CUED_REC_WORD,1.652297e+12,0,<>,syringe,-999,dentist,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,-1,NaN,NaN,NaN,NaN
12576,CUED_REC_WORD,1.652297e+12,0,pastries,croissant,2,bakery,cafe,"(36.74, -1.91, 59.06)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,153,NaN,True,1.652297e+12,NaN


In [85]:
a.at[21472, "keypress_code"] = [1,2]
a

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress_code,keypress,keypress_mstime,keypress_keycode
11955,CUED_REC_WORD,1.652297e+12,0,decals,decals,8,bike shop,bike shop,"(14.49, 2.27, -33.74)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,66,NaN,NaN,NaN,NaN
12025,CUED_REC_WORD,1.652297e+12,0,pastries,pastries,2,cafe,cafe,"(36.74, -1.91, 59.06)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,153,NaN,True,1.652297e+12,NaN
12092,CUED_REC_WORD,1.652297e+12,0,anchovies,anchovies,3,pizzeria,pizzeria,"(-14.25, 0.00, -12.89)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,6,NaN,NaN,NaN,NaN
12197,CUED_REC_WORD,1.652297e+12,0,<>,watch,-999,jewelry store,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,-1,NaN,NaN,NaN,NaN
12225,CUED_REC_WORD,1.652297e+12,0,guinea_pig,guinea_pig,12,pet store,pet store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,96,NaN,NaN,NaN,NaN
12289,CUED_REC_WORD,1.652297e+12,0,floor_mat,floor_mat,7,gym,gym,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,85,NaN,True,1.652297e+12,NaN
12371,CUED_REC_WORD,1.652297e+12,0,markers,markers,9,craft shop,craft shop,"(35.40, 0.00, -33.11)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,128,NaN,NaN,NaN,NaN
12475,CUED_REC_WORD,1.652297e+12,0,<>,towels,-999,barber shop,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,-1,NaN,NaN,NaN,NaN
12543,CUED_REC_WORD,1.652297e+12,0,<>,syringe,-999,dentist,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,-1,NaN,NaN,NaN,NaN
12576,CUED_REC_WORD,1.652297e+12,0,pastries,croissant,2,bakery,cafe,"(36.74, -1.91, 59.06)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT3,0,-999,-999,153,NaN,True,1.652297e+12,NaN


In [66]:
    ###############################################################################################
    # add EFR presses
    ###############################################################################################
    tmp["keypress_code"] = np.nan
    tmp = tmp.astype({"keypress_code":object})
    
    # FREE RECALL
    rec_evs = tmp.query('type == "REC_WORD" | type == "REC_STOP"')
    rec_evs_index = rec_evs.index

    for i in range(len(rec_evs)-1):

        curr_row = rec_evs.iloc[i]
        if curr_row.type != "REC_WORD":
            continue
        next_row = rec_evs.iloc[i+1]

        curr_time = curr_row.mstime
        next_time = next_row.mstime

        EFR_candidates = tmp.query('(type == "keypress" | type == "key press/release") & \
                                   mstime > @curr_time & mstime < @next_time')
        if len(EFR_candidates.query('type == "keypress"')) > 0:
            responses = EFR_candidates["key code"].unique()
            responses = responses[~np.isnan(responses)]
            response_time = EFR_candidates.query('type == "keypress"').mstime.values[0]

            tmp.at[rec_evs_index[i], "keypress"] = True
            tmp.at[rec_evs_index[i], "keypress_code"] = responses.tolist()
            tmp.at[rec_evs_index[i], "keypress_mstime"] = response_time

    # CUED RECALL
    cued_rec_evs = tmp.query('type == "CUED_REC_WORD" | type == "CUED_REC_STOP"')
    cued_rec_evs_index = cued_rec_evs.index

    for i in range(len(cued_rec_evs)-1):

        curr_row = cued_rec_evs.iloc[i]
        if curr_row.type != "CUED_REC_WORD":
            continue
        next_row = cued_rec_evs.iloc[i+1]

        curr_time = curr_row.mstime
        next_time = next_row.mstime

        EFR_candidates = tmp.query('(type == "keypress" | type == "key press/release") & \
                                    mstime > @curr_time & mstime < @next_time')
        if len(EFR_candidates.query('type == "keypress"')) > 0:
            responses = EFR_candidates["key code"].unique()
            responses = responses[~np.isnan(responses)]
            response_time = EFR_candidates.query('type == "keypress"').mstime.values[0]

            tmp.at[cued_rec_evs_index[i], "keypress"] = True
            tmp.at[cued_rec_evs_index[i], "keypress_code"] = responses.tolist()
            tmp.at[cued_rec_evs_index[i], "keypress_mstime"] = response_time

    # FINAL STORE RECALL
    store_rec_evs = tmp.query('type == "FSR_REC_WORD" | type == "FSR_REC_STOP"')
    store_rec_evs_index = store_rec_evs.index

    for i in range(len(store_rec_evs)-1):

        curr_row = store_rec_evs.iloc[i]
        if curr_row.type != "FSR_REC_WORD":
            continue
        next_row = store_rec_evs.iloc[i+1]

        curr_time = curr_row.mstime
        next_time = next_row.mstime

        EFR_candidates = tmp.query('(type == "keypress" | type == "key press/release") & \
                                    mstime > @curr_time & mstime < @next_time')
        if len(EFR_candidates.query('type == "keypress"')) > 0:
            responses = EFR_candidates["key code"].unique()
            responses = responses[~np.isnan(responses)]
            response_time = EFR_candidates.query('type == "keypress"').mstime.values[0]

            tmp.at[store_rec_evs_index[i], "keypress"] = True
            tmp.at[store_rec_evs_index[i], "keypress_code"] = responses.tolist()
            tmp.at[store_rec_evs_index[i], "keypress_mstime"] = response_time

    # FINAL FREE RECALL
    final_rec_evs = tmp.query('type == "FFR_REC_WORD" | type == "FFR_REC_STOP"')
    final_rec_evs_index = final_rec_evs.index

    for i in range(len(final_rec_evs)-1):

        curr_row = final_rec_evs.iloc[i]
        if curr_row.type != "FFR_REC_WORD":
            continue
        next_row = final_rec_evs.iloc[i+1]

        curr_time = curr_row.mstime
        next_time = next_row.mstime

        EFR_candidates = tmp.query('(type == "keypress" | type == "key press/release") & \
                                    mstime > @curr_time & mstime < @next_time')
        if len(EFR_candidates.query('type == "keypress"')) > 0:
            responses = EFR_candidates["key code"].unique()
            responses = responses[~np.isnan(responses)]
            response_time = EFR_candidates.query('type == "keypress"').mstime.values[0]

            tmp.at[final_rec_evs_index[i], "keypress"] = True
            tmp.at[final_rec_evs_index[i], "keypress_code"] = responses.tolist()
            tmp.at[final_rec_evs_index[i], "keypress_mstime"] = response_time

['/scratch/new_courier_pilot/PILOT1/session_0', '/scratch/new_courier_pilot/PILOT2/session_0', '/scratch/new_courier_pilot/PILOT3/session_0', '/scratch/new_courier_pilot/PILOT4/session_1', '/scratch/new_courier_pilot/PILOT4/session_0']
subject PILOT1, sess 0
subject PILOT2, sess 0
subject PILOT3, sess 0
subject PILOT4, sess 1
subject PILOT4, sess 0


In [68]:
tmp.query('type == "REC_WORD"')

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress_code,keypress,keypress_mstime
11728,REC_WORD,1.652717e+12,0,x-ray_machine,NaN,12,NaN,dentist,"(32.20, 0.33, -60.90)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,235,NaN,NaN,NaN
11741,REC_WORD,1.652717e+12,0,mistletoes,NaN,-999,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,0,-1,NaN,NaN,NaN
11756,REC_WORD,1.652717e+12,0,gauze,NaN,8,NaN,pharmacy,"(-9.09, 0.00, 17.86)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,92,NaN,NaN,NaN
11821,REC_WORD,1.652717e+12,0,scissors,NaN,-999,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,186,[98.0],True,1.652717e+12
11848,REC_WORD,1.652717e+12,0,milk,NaN,-999,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,132,[98.0],True,1.652717e+12
11873,REC_WORD,1.652717e+12,0,chocolate_chips,NaN,11,NaN,bakery,"(59.29, 4.65, -9.16)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,47,NaN,NaN,NaN
11953,REC_WORD,1.652717e+12,0,chocolate_chips,NaN,11,NaN,bakery,"(59.29, 4.65, -9.16)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,47,[98.0],True,1.652717e+12
12126,REC_WORD,1.652717e+12,0,menu,NaN,4,NaN,pizzeria,"(-14.25, 0.00, -12.89)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,130,NaN,NaN,NaN
12194,REC_WORD,1.652717e+12,0,inflatable,NaN,-999,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,0,-1,[98.0],True,1.652717e+12
45516,REC_WORD,1.652722e+12,1,training_wheels,NaN,12,NaN,bike shop,"(14.49, 2.27, -33.74)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT4,0,-999,-1,225,NaN,NaN,NaN


In [45]:
data = []
sub = "PILOT2"

data_dir = "/scratch/new_courier_pilot/"

for line in open("/scratch/new_courier_pilot/{}/session_0/session.jsonl".format(sub), "r"):
    
    if '"point condition":SerialPosition,' in line:
        line = line.replace('"point condition":SerialPosition,', '')
    elif '"point condition":SpatialPosition,' in line:
        line = line.replace('"point condition":SpatialPosition,', '')
    elif '"point condition":Random,' in line:
        line = line.replace('"point condition":Random,', '')
    
    data_dict = json.loads(line)
    
    data.append(data_dict)

for d in data:
    for k,v in d["data"].items():
        d[k] = v

df = pd.DataFrame(data)

In [46]:
column_names = ["type", "time", "trial number", "item name", "item", "serial position", "store", "store name", "store position", 
                "player position", "positionX", "positionZ", "correct direction (degrees)", "pointed direction (degrees)", "response"]
df = df[column_names]
df = df.rename(columns={
                        'trial number': 'trial',
                        'time': 'mstime',
                        'serial position': 'serialpos',
                        'item': 'cued_item',
                        'store': 'cued_store',
                        'store name': 'store',
                        'item name': 'item'
                        })
df["subject"] = sub

# change the type name to match lab's data frame formatting
df = df.replace({'type': {'object presentation begins': 'WORD', 
                     'free recall': 'REC_WORD',
                     'start free recall': 'REC_START',
                     'end free recall': 'REC_STOP',
                     'start cued recall typing': 'CUED_REC_CUE',
                     'cued recall': 'CUED_REC_WORD',
                     'final store recall': 'SR_REC_WORD', 
                     'final object recall': 'FFR_REC_WORD'
                    }})

# Free Recall Processing

In [49]:
# now get recall dataframe ready to insert
recall_df = pd.DataFrame(columns = df.columns)

count = 0
rec_starts = df.query('type == "REC_START"')
for i, row in rec_starts.iterrows():
    sub = rec_starts['subject'].unique()[0]
    
    if row["type"] == "REC_START":
        # first REC_START will indicate practice recall stage
        if count > 0:
            rec_start_time = row.mstime
            
            # let's load appropriate .ann files
            data_dir = "/scratch/new_courier_pilot/{}/session_0/{}.ann".format(sub, count-1)
            with open(data_dir, "r") as file:
                lines = file.readlines()
                for line in lines:
                    if line[0] != "#":
                        recall_info = line.strip("\n").split("\t")
                        
                        if len(recall_info) == 3:
                            rectime = float(recall_info[0]) + rec_start_time
                            item = recall_info[2].lower()
                            itemno = recall_info[1]
                            
                            recall_df = recall_df.append({"type":"REC_WORD", "mstime":rectime, "trial":count-1, 
                                                          "item":item, "itemno":itemno, "subject":sub}, ignore_index=True)
    
        count += 1
        
recall_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,REC_WORD,1.649794e+12,0,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,77
1,REC_WORD,1.649794e+12,0,hula_hoop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113
2,REC_WORD,1.649794e+12,0,rope,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,178
3,REC_WORD,1.649794e+12,0,locks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124
4,REC_WORD,1.649794e+12,0,saxophone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,183
5,REC_WORD,1.649794e+12,0,cotton_balls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,58
6,REC_WORD,1.649794e+12,0,sweet-n-low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210
7,REC_WORD,1.649794e+12,0,head_band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,110
8,REC_WORD,1.649794e+12,0,muffins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,136
9,REC_WORD,1.649794e+12,0,bracelet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,28


In [50]:
# then, fill out the blanks
recall_df["store"] = recall_df["store"].astype(str)
recall_df["store position"] = recall_df["store position"].astype(str)

for i, row in recall_df.iterrows():
    word_evs = df[(df.trial == row.trial) & (df.type == "WORD")]
    
    row_item = row["item"]
    recall_word = word_evs.query('item == @row_item')
    
    if len(recall_word) != 0:        
        serialpos = recall_word["serialpos"].values[0]
        store = recall_word["store"].values[0]
        store_position = recall_word["store position"].values[0]
        
        recall_df.at[i, "serialpos"] = serialpos
        recall_df.at[i, "store"] = store
        recall_df.at[i, "store position"] = store_position
        
    else:
        recall_df.at[i, "serialpos"] = -1
    
recall_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,REC_WORD,1.649794e+12,0,eggs,NaN,-1.0,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,77
1,REC_WORD,1.649794e+12,0,hula_hoop,NaN,9.0,NaN,toy store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113
2,REC_WORD,1.649794e+12,0,rope,NaN,11.0,NaN,gym,"(-38.95, 0.00, -61.87)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,178
3,REC_WORD,1.649794e+12,0,locks,NaN,10.0,NaN,bike shop,"(-31.82, 4.70, 55.71)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124
4,REC_WORD,1.649794e+12,0,saxophone,NaN,3.0,NaN,music store,"(-29.57, 1.89, -37.24)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,183
5,REC_WORD,1.649794e+12,0,cotton_balls,NaN,5.0,NaN,dentist,"(14.55, -0.04, -58.79)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,58
6,REC_WORD,1.649794e+12,0,sweet-n-low,NaN,2.0,NaN,cafe,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210
7,REC_WORD,1.649794e+12,0,head_band,NaN,4.0,NaN,clothing store,"(-60.00, 0.00, -11.50)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,110
8,REC_WORD,1.649794e+12,0,muffins,NaN,-1.0,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,136
9,REC_WORD,1.649794e+12,0,bracelet,NaN,8.0,NaN,jewelry store,"(62.34, 0.00, -33.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,28


# Cued Recall Processing

In [51]:
cued_rec_start = df.query('type == "start cued recall"').mstime.values[1]   ## skip practice cued recall
cued_rec_events = df.query('type == "cued recall recording start" & mstime >= @cued_rec_start')

cued_recall_df = pd.DataFrame(columns = df.columns)

for i, row in cued_rec_events.iterrows():
    sub = row.subject
    trial = int(row.trial)
    cued_item = row.cued_item
    cued_store = row.cued_store
    cued_time = row.mstime
    
    word_evs = df.query('type == "WORD" & subject == @sub & trial == @trial')
    stores = word_evs.store.unique()
    
    if cued_store not in stores:
        continue
    
    # locate the .ann file
    data_dir = "/scratch/new_courier_pilot/{}/session_0/{}-{}.ann".format(sub, trial, cued_store)
    with open(data_dir, "r") as file:
        lines = file.readlines()
        for line in lines:
            if line[0] != "#":
                recall_info = line.strip("\n").split("\t")
                
                if len(recall_info) == 3:
                    rectime = float(recall_info[0]) + cued_time
                    item = recall_info[2].lower()
                    itemno = recall_info[1]
                    
                    cued_recall_df = cued_recall_df.append({"type": "CUED_REC_WORD", "mstime": rectime, "trial": trial, 
                                                            "item": item, "itemno": itemno, "cued_item": cued_item, "cued_store": cued_store, 
                                                            "subject":sub}, ignore_index=True)
    
cued_recall_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,CUED_REC_WORD,1.649794e+12,0,sweet-n-low,sweet-n-low,NaN,cafe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210
1,CUED_REC_WORD,1.649794e+12,0,locks,locks,NaN,bike shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124
2,CUED_REC_WORD,1.649794e+12,0,rope,rope,NaN,gym,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,178
3,CUED_REC_WORD,1.649794e+12,0,cotton_balls,cotton_balls,NaN,dentist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,58
4,CUED_REC_WORD,1.649794e+12,0,head_band,head_band,NaN,clothing store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,110
5,CUED_REC_WORD,1.649794e+12,0,mice,mice,NaN,pet store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,131
6,CUED_REC_WORD,1.649794e+12,0,<>,cough_syrup,NaN,pharmacy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,-1
7,CUED_REC_WORD,1.649794e+12,0,<>,yarn,NaN,craft shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,-1
8,CUED_REC_WORD,1.649794e+12,0,hula_hoop,hula_hoop,NaN,toy store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113
9,CUED_REC_WORD,1.649794e+12,0,bracelet,bracelet,NaN,jewelry store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,28


In [52]:
# then, fill out the blanks
cued_recall_df["store"] = cued_recall_df["store"].astype(str)
cued_recall_df["store position"] = cued_recall_df["store position"].astype(str)

for i, row in cued_recall_df.iterrows():
    word_evs = df[(df.trial == row.trial) & (df.type == "WORD")]
    
    row_item = row["item"]
    recall_word = word_evs.query('item == @row_item')
    
    if len(recall_word) != 0:        
        serialpos = recall_word["serialpos"].values[0]
        store = recall_word["store"].values[0]
        store_position = recall_word["store position"].values[0]
        
        cued_recall_df.at[i, "serialpos"] = serialpos
        cued_recall_df.at[i, "store"] = store
        cued_recall_df.at[i, "store position"] = store_position
        
    else:
        cued_recall_df.at[i, "serialpos"] = -1
    
cued_recall_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,CUED_REC_WORD,1.649794e+12,0,sweet-n-low,sweet-n-low,2.0,cafe,cafe,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210
1,CUED_REC_WORD,1.649794e+12,0,locks,locks,10.0,bike shop,bike shop,"(-31.82, 4.70, 55.71)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124
2,CUED_REC_WORD,1.649794e+12,0,rope,rope,11.0,gym,gym,"(-38.95, 0.00, -61.87)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,178
3,CUED_REC_WORD,1.649794e+12,0,cotton_balls,cotton_balls,5.0,dentist,dentist,"(14.55, -0.04, -58.79)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,58
4,CUED_REC_WORD,1.649794e+12,0,head_band,head_band,4.0,clothing store,clothing store,"(-60.00, 0.00, -11.50)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,110
5,CUED_REC_WORD,1.649794e+12,0,mice,mice,1.0,pet store,pet store,"(32.20, 0.33, -60.90)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,131
6,CUED_REC_WORD,1.649794e+12,0,<>,cough_syrup,-1.0,pharmacy,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,-1
7,CUED_REC_WORD,1.649794e+12,0,<>,yarn,-1.0,craft shop,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,-1
8,CUED_REC_WORD,1.649794e+12,0,hula_hoop,hula_hoop,9.0,toy store,toy store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113
9,CUED_REC_WORD,1.649794e+12,0,bracelet,bracelet,8.0,jewelry store,jewelry store,"(62.34, 0.00, -33.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,28


# Final Free Recall Processing

In [53]:
FFR_store_df = pd.DataFrame(columns = df.columns)
FFR_store_start = df.query('type == "final store recall recording start"').mstime.values[0]

# let's load appropriate .ann files
data_dir = "/scratch/new_courier_pilot/{}/session_0/final store-0.ann".format(sub, count-1)
with open(data_dir, "r") as file:
    lines = file.readlines()
    for line in lines:
        if line[0] != "#":
            recall_info = line.strip("\n").split("\t")

            if len(recall_info) == 3:
                rectime = float(recall_info[0]) + FFR_store_start
                item = recall_info[2].lower()
                itemno = recall_info[1]
                
                FFR_store_df = FFR_store_df.append({"type":"FFR_STORE", "mstime":rectime, 
                                                    "item":item, "itemno":itemno, "subject":sub}, ignore_index=True)

FFR_store_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,FFR_STORE,1.649795e+12,NaN,hardware store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,11
1,FFR_STORE,1.649795e+12,NaN,grocery store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,9
2,FFR_STORE,1.649795e+12,NaN,clothing store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,5
3,FFR_STORE,1.649795e+12,NaN,cafe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,4
4,FFR_STORE,1.649795e+12,NaN,bakery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,1
5,FFR_STORE,1.649795e+12,NaN,florist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,8
6,FFR_STORE,1.649795e+12,NaN,dentist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,7
7,FFR_STORE,1.649795e+12,NaN,pet store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,14
8,FFR_STORE,1.649795e+12,NaN,toy store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,17
9,FFR_STORE,1.649795e+12,NaN,craft shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,6


In [54]:
FFR_item_df = pd.DataFrame(columns = df.columns)
FFR_item_start = df.query('type == "final object recall recording start"').mstime.values[0]

# let's load appropriate .ann files
data_dir = "/scratch/new_courier_pilot/{}/session_0/final free-0.ann".format(sub, count-1)
with open(data_dir, "r") as file:
    lines = file.readlines()
    for line in lines:
        if line[0] != "#":
            recall_info = line.strip("\n").split("\t")

            if len(recall_info) == 3:
                rectime = float(recall_info[0]) + FFR_item_start
                item = recall_info[2].lower()
                itemno = recall_info[1]
                
                FFR_item_df = FFR_item_df.append({"type":"FFR_ITEM", "mstime":rectime, 
                                                  "item":item, "itemno":itemno, "subject":sub}, ignore_index=True)

FFR_item_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,FFR_ITEM,1.649795e+12,NaN,ribbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,175
1,FFR_ITEM,1.649795e+12,NaN,roll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,177
2,FFR_ITEM,1.649795e+12,NaN,saxophone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,183
3,FFR_ITEM,1.649795e+12,NaN,sweet-n-low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210
4,FFR_ITEM,1.649795e+12,NaN,soil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,197
5,FFR_ITEM,1.649795e+12,NaN,locks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124
6,FFR_ITEM,1.649795e+12,NaN,screws,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,189
7,FFR_ITEM,1.649795e+12,NaN,stuffed_animal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,204
8,FFR_ITEM,1.649795e+12,NaN,hula_hoop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113
9,FFR_ITEM,1.649795e+12,NaN,contact_solution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,56


In [55]:
# then, fill out the blanks
FFR_item_df["store"] = FFR_item_df["store"].astype(str)
FFR_item_df["store position"] = FFR_item_df["store position"].astype(str)

for i, row in FFR_item_df.iterrows():
    word_evs = df.query('type == "WORD"')
    
    row_item = row["item"]
    recall_word = word_evs.query('item == @row_item')
    
    if len(recall_word) != 0:        
        serialpos = recall_word["serialpos"].values[0]
        store = recall_word["store"].values[0]
        store_position = recall_word["store position"].values[0]
        
        FFR_item_df.at[i, "serialpos"] = serialpos
        FFR_item_df.at[i, "store"] = store
        FFR_item_df.at[i, "store position"] = store_position
        
    else:
        FFR_item_df.at[i, "serialpos"] = -1
    
FFR_item_df

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno
0,FFR_ITEM,1.649795e+12,NaN,ribbon,NaN,11.0,NaN,craft shop,"(59.29, 4.65, -9.16)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,175
1,FFR_ITEM,1.649795e+12,NaN,roll,NaN,4.0,NaN,bakery,"(14.49, 2.27, -33.74)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,177
2,FFR_ITEM,1.649795e+12,NaN,saxophone,NaN,3.0,NaN,music store,"(-29.57, 1.89, -37.24)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,183
3,FFR_ITEM,1.649795e+12,NaN,sweet-n-low,NaN,2.0,NaN,cafe,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210
4,FFR_ITEM,1.649795e+12,NaN,soil,NaN,10.0,NaN,florist,"(36.74, -1.91, 59.06)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,197
5,FFR_ITEM,1.649795e+12,NaN,locks,NaN,10.0,NaN,bike shop,"(-31.82, 4.70, 55.71)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124
6,FFR_ITEM,1.649795e+12,NaN,screws,NaN,7.0,NaN,hardware store,"(35.06, 0.00, 26.66)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,189
7,FFR_ITEM,1.649795e+12,NaN,stuffed_animal,NaN,6.0,NaN,toy store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,204
8,FFR_ITEM,1.649795e+12,NaN,hula_hoop,NaN,9.0,NaN,toy store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113
9,FFR_ITEM,1.649795e+12,NaN,contact_solution,NaN,1.0,NaN,pharmacy,"(-14.25, 0.00, -12.89)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,56


# Add them up

In [56]:
# now put it in right place
tmp = df.copy()
tmp = tmp.append(recall_df)
tmp = tmp.append(cued_recall_df)
tmp = tmp.append(FFR_store_df)
tmp = tmp.append(FFR_item_df)
tmp = tmp.sort_values('mstime').reset_index(drop=True)

# add "recalled" column

In [57]:
with open('/home1/leochang/ELEMEM_PILOT/all_items.txt', "r") as f:
    wordpool = [w.strip() for w in f.readlines()]
with open('/data/behavioral/mturk/dictionary.txt', 'r') as f:
    dictionary = [w.strip().replace("_", " ") for w in f.readlines()]

In [ ]:
tmp = add_recalled(tmp)
tmp.query('type == "WORD" | type == "REC_WORD"')

# Add "session" column

In [60]:
tmp["session"] = 0
tmp

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno,recalled,session
0,versions,1.649792e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,NaN,NaN,0
1,experimentConfig,1.649792e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,NaN,NaN,0
2,systemConfig,1.649792e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,NaN,NaN,0
3,PlayerTransform,1.649792e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.536743e-07,-1.364242e-12,NaN,NaN,NaN,PILOT2,NaN,NaN,0
4,store mappings,1.649792e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18535,PlayerTransform,1.649795e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.327605e+01,-4.904773e+01,NaN,NaN,NaN,PILOT2,NaN,NaN,0
18536,PlayerTransform,1.649795e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.327605e+01,-4.904773e+01,NaN,NaN,NaN,PILOT2,NaN,NaN,0
18537,PlayerTransform,1.649795e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.327605e+01,-4.904773e+01,NaN,NaN,NaN,PILOT2,NaN,NaN,0
18538,PlayerTransform,1.649795e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.327605e+01,-4.904773e+01,NaN,NaN,NaN,PILOT2,NaN,NaN,0


# Add "itemno" values to WORD events

In [61]:
def get_item_id(item, wordpool):
    if item not in wordpool:
        return -1
    return wordpool.index(item)+1

def add_itemno(events, wordpool):
    events.loc[(events["type"] == 'WORD'), "itemno"] = \
    events.loc[(events["type"] == 'WORD'), "item"].apply(lambda x: get_item_id(x, wordpool))

    return events
                                                    
tmp = add_itemno(tmp, wordpool)
tmp.query('type == "WORD" | type == "REC_WORD"')

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno,recalled,session
7565,WORD,1.649793e+12,0.0,mice,NaN,1.0,NaN,pet store,"(32.20, 0.33, -60.90)","(31.26, 0.00, -49.05)",NaN,NaN,NaN,NaN,NaN,PILOT2,131,0.0,0
7888,WORD,1.649793e+12,0.0,sweet-n-low,NaN,2.0,NaN,cafe,"(58.31, 4.05, 27.93)","(49.46, 0.00, 28.18)",NaN,NaN,NaN,NaN,NaN,PILOT2,210,1.0,0
8176,WORD,1.649793e+12,0.0,saxophone,NaN,3.0,NaN,music store,"(-29.57, 1.89, -37.24)","(-28.74, 0.00, -46.53)",NaN,NaN,NaN,NaN,NaN,PILOT2,183,1.0,0
8545,WORD,1.649794e+12,0.0,head_band,NaN,4.0,NaN,clothing store,"(-60.00, 0.00, -11.50)","(-48.87, 0.00, -10.78)",NaN,NaN,NaN,NaN,NaN,PILOT2,110,1.0,0
8724,WORD,1.649794e+12,0.0,cotton_balls,NaN,5.0,NaN,dentist,"(14.55, -0.04, -58.79)","(9.57, 0.00, -48.95)",NaN,NaN,NaN,NaN,NaN,PILOT2,58,1.0,0
9085,WORD,1.649794e+12,0.0,yarn,NaN,6.0,NaN,craft shop,"(59.29, 4.65, -9.16)","(49.50, 0.00, -8.45)",NaN,NaN,NaN,NaN,NaN,PILOT2,237,0.0,0
9251,WORD,1.649794e+12,0.0,cough_syrup,NaN,7.0,NaN,pharmacy,"(-14.25, 0.00, -12.89)","(-0.87, 0.00, -9.18)",NaN,NaN,NaN,NaN,NaN,PILOT2,59,0.0,0
9413,WORD,1.649794e+12,0.0,bracelet,NaN,8.0,NaN,jewelry store,"(62.34, 0.00, -33.51)","(50.37, 0.00, -32.72)",NaN,NaN,NaN,NaN,NaN,PILOT2,28,1.0,0
9541,WORD,1.649794e+12,0.0,hula_hoop,NaN,9.0,NaN,toy store,"(-3.07, 0.00, -58.51)","(-6.36, 0.00, -49.47)",NaN,NaN,NaN,NaN,NaN,PILOT2,113,1.0,0
9838,WORD,1.649794e+12,0.0,locks,NaN,10.0,NaN,bike shop,"(-31.82, 4.70, 55.71)","(-29.23, 0.00, 49.50)",NaN,NaN,NaN,NaN,NaN,PILOT2,124,1.0,0


In [62]:
tmp.query('type == "WORD"')

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno,recalled,session
7565,WORD,1.649793e+12,0.0,mice,NaN,1.0,NaN,pet store,"(32.20, 0.33, -60.90)","(31.26, 0.00, -49.05)",NaN,NaN,NaN,NaN,NaN,PILOT2,131,0.0,0
7888,WORD,1.649793e+12,0.0,sweet-n-low,NaN,2.0,NaN,cafe,"(58.31, 4.05, 27.93)","(49.46, 0.00, 28.18)",NaN,NaN,NaN,NaN,NaN,PILOT2,210,1.0,0
8176,WORD,1.649793e+12,0.0,saxophone,NaN,3.0,NaN,music store,"(-29.57, 1.89, -37.24)","(-28.74, 0.00, -46.53)",NaN,NaN,NaN,NaN,NaN,PILOT2,183,1.0,0
8545,WORD,1.649794e+12,0.0,head_band,NaN,4.0,NaN,clothing store,"(-60.00, 0.00, -11.50)","(-48.87, 0.00, -10.78)",NaN,NaN,NaN,NaN,NaN,PILOT2,110,1.0,0
8724,WORD,1.649794e+12,0.0,cotton_balls,NaN,5.0,NaN,dentist,"(14.55, -0.04, -58.79)","(9.57, 0.00, -48.95)",NaN,NaN,NaN,NaN,NaN,PILOT2,58,1.0,0
9085,WORD,1.649794e+12,0.0,yarn,NaN,6.0,NaN,craft shop,"(59.29, 4.65, -9.16)","(49.50, 0.00, -8.45)",NaN,NaN,NaN,NaN,NaN,PILOT2,237,0.0,0
9251,WORD,1.649794e+12,0.0,cough_syrup,NaN,7.0,NaN,pharmacy,"(-14.25, 0.00, -12.89)","(-0.87, 0.00, -9.18)",NaN,NaN,NaN,NaN,NaN,PILOT2,59,0.0,0
9413,WORD,1.649794e+12,0.0,bracelet,NaN,8.0,NaN,jewelry store,"(62.34, 0.00, -33.51)","(50.37, 0.00, -32.72)",NaN,NaN,NaN,NaN,NaN,PILOT2,28,1.0,0
9541,WORD,1.649794e+12,0.0,hula_hoop,NaN,9.0,NaN,toy store,"(-3.07, 0.00, -58.51)","(-6.36, 0.00, -49.47)",NaN,NaN,NaN,NaN,NaN,PILOT2,113,1.0,0
9838,WORD,1.649794e+12,0.0,locks,NaN,10.0,NaN,bike shop,"(-31.82, 4.70, 55.71)","(-29.23, 0.00, 49.50)",NaN,NaN,NaN,NaN,NaN,PILOT2,124,1.0,0


# Add "intrusion" column

In [94]:
def add_intrusion(events):
    '''
    uses REC_WORD and WORD events to determine if a recalled word is a PLI/XLI
    '''
    events = events.sort_values("mstime")

    def check_list(row):

        presentation = events[(events["type"] == 'WORD') \
                              & (events["itemno"] == row["itemno"]) \
                              & (events["trial"] <= row["trial"])]

        if len(presentation.index) == 0:
            return -1
        else:
            # also captures repeated presentations
            list_delta = row["trial"] - presentation.iloc[-1]["trial"]
            # list_delta = list_delta.values[0] # Series are annoying

            return -1 if list_delta < 0 else list_delta

    events.loc[events["type"] == 'REC_WORD', "intrusion"] = events[events["type"] == 'REC_WORD'].apply(check_list, axis=1)

    return events

tmp[["trial", "serialpos", "recalled", "itemno", "intrusion"]] = tmp[["trial", "serialpos", "recalled", "itemno", "intrusion"]].fillna(-999)
tmp = tmp.astype({"trial":int, "serialpos":int, "recalled":int, "itemno":int, "intrusion":int})
tmp = add_intrusion(tmp)
tmp.query('type == "REC_WORD"')

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,subject,itemno,recalled,session,intrusion
10471,REC_WORD,1.649794e+12,0,eggs,NaN,-1,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,77,-999,0,-1
10481,REC_WORD,1.649794e+12,0,hula_hoop,NaN,9,NaN,toy store,"(-3.07, 0.00, -58.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,113,-999,0,0
10488,REC_WORD,1.649794e+12,0,rope,NaN,11,NaN,gym,"(-38.95, 0.00, -61.87)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,178,-999,0,0
10499,REC_WORD,1.649794e+12,0,locks,NaN,10,NaN,bike shop,"(-31.82, 4.70, 55.71)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,124,-999,0,0
10510,REC_WORD,1.649794e+12,0,saxophone,NaN,3,NaN,music store,"(-29.57, 1.89, -37.24)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,183,-999,0,0
10522,REC_WORD,1.649794e+12,0,cotton_balls,NaN,5,NaN,dentist,"(14.55, -0.04, -58.79)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,58,-999,0,0
10532,REC_WORD,1.649794e+12,0,sweet-n-low,NaN,2,NaN,cafe,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,210,-999,0,0
10548,REC_WORD,1.649794e+12,0,head_band,NaN,4,NaN,clothing store,"(-60.00, 0.00, -11.50)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,110,-999,0,0
10606,REC_WORD,1.649794e+12,0,muffins,NaN,-1,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,136,-999,0,-1
10661,REC_WORD,1.649794e+12,0,bracelet,NaN,8,NaN,jewelry store,"(62.34, 0.00, -33.51)",NaN,NaN,NaN,NaN,NaN,NaN,PILOT2,28,-999,0,0


In [177]:
tmp.to_csv("{}.csv".format(sub))

In [51]:
# FREE RECALL
rec_evs = tmp.query('type == "REC_WORD" | type == "REC_STOP"')
rec_evs_index = rec_evs.index

for i in range(len(rec_evs)-1):

    curr_row = rec_evs.iloc[i]
    if curr_row.type != "REC_WORD":
        continue
    next_row = rec_evs.iloc[i+1]

    curr_time = curr_row.mstime
    next_time = next_row.mstime

    EFR_candidates = tmp.query('(type == "keypress" | type == "key press/release") & mstime > @curr_time & mstime < @next_time')
    display(EFR_candidates)
    if len(EFR_candidates.query('type == "keypress"')) > 0:
        responses = EFR_candidates["key code"].unique()
        responses = responses[~np.isnan(responses)]
        print(responses)
        
        response_time = EFR_candidates.query('type == "keypress"').mstime.values[0]
        print(response_time)
    
        tmp.at[rec_evs_index[i], "keypress"] = True
        tmp.at[rec_evs_index[i], "keypress_code"] = responses
        tmp.at[rec_evs_index[i], "keypress_mstime"] = response_time
        
        break
        

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress,keypress_mstime


,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress,keypress_mstime


,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress,keypress_mstime


,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress,keypress_mstime


,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress,keypress_mstime
10359,keypress,1.649776e+12,-999,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,reject button,NaN,PILOT1,0,-999,-999,-999,NaN,NaN
10360,key press/release,1.649776e+12,-999,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,PILOT1,0,-999,-999,-999,NaN,NaN
10363,key press/release,1.649776e+12,-999,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,PILOT1,0,-999,-999,-999,NaN,NaN


[98.]
1649776105647.93


In [52]:
tmp.query('type == "REC_WORD"')

,type,mstime,trial,item,cued_item,serialpos,cued_store,store,store position,player position,positionX,positionZ,correct direction (degrees),pointed direction (degrees),response,key code,subject,session,recalled,intrusion,itemno,keypress,keypress_mstime,keypress_code
10286,REC_WORD,1.649776e+12,0,chocolate_chips,NaN,12,NaN,bakery,"(62.34, 0.00, -33.51)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,47,NaN,NaN,NaN
10300,REC_WORD,1.649776e+12,0,hamster,NaN,10,NaN,pet store,"(-38.95, 0.00, -61.87)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,105,NaN,NaN,NaN
10313,REC_WORD,1.649776e+12,0,helmet,NaN,9,NaN,bike shop,"(35.06, 0.00, 26.66)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,111,NaN,NaN,NaN
10325,REC_WORD,1.649776e+12,0,socks,NaN,6,NaN,clothing store,"(32.20, 0.33, -60.90)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,196,NaN,NaN,NaN
10342,REC_WORD,1.649776e+12,0,straws,NaN,7,NaN,cafe,"(26.49, 0.58, 16.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,203,True,1.649776e+12,98.0
10483,REC_WORD,1.649776e+12,0,socks,NaN,6,NaN,clothing store,"(32.20, 0.33, -60.90)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,196,True,NaN,NaN
10512,REC_WORD,1.649776e+12,0,helmet,NaN,9,NaN,bike shop,"(35.06, 0.00, 26.66)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,111,True,NaN,NaN
14382,REC_WORD,1.649777e+12,1,ferns,NaN,12,NaN,florist,"(58.31, 4.05, 27.93)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,82,NaN,NaN,NaN
14394,REC_WORD,1.649777e+12,1,cookies,NaN,11,NaN,bakery,"(62.34, 0.00, -33.51)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,57,NaN,NaN,NaN
14406,REC_WORD,1.649777e+12,1,computer,NaN,1,NaN,dentist,"(-14.25, 0.00, -12.89)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PILOT1,0,-999,-1,53,NaN,NaN,NaN
